# Построение и обучение модели PINN для решения двумерного уравнения теплопроводности

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

import onnx
import json

import netron

import matplotlib.pyplot as plt
from IPython.display import IFrame

### Определение аналитического решения

Для проверки обученной модели получем функцию аналитического решения. Она позволит как посчитать ошибку по $ L^2 $ норме в конкретный момент времени, так и провести сравнительный анализ эволюции решения полученного с помощью PINN и аналитическим решением.

In [3]:
def analytical_solution(x: np.ndarray, 
                        y: np.ndarray, 
                        t: np.ndarray,
                        bounds: dict,
                        alpha: float) -> np.ndarray:
    """
    Аналитическое решение

    Параметры:
        x: np.ndarray
            Вектор координат x.
        y: np.ndarray
            Вектор координат y.
        t: np.ndarray
            Вектор координат t.
        bounds: dict
            Границы расчетной области.
        alpha: float
            Коэффициент температурапроводности материала.

    Возвращает:
        np.ndarray
            Вектор значений температур, являющихся аналитическим решением.
    """
    u = (np.sin(np.pi * x/bounds['x']) * np.sin(np.pi * y/bounds['y'])
         * np.exp(-2 * np.pi**2 * alpha * t/(bounds['x']*bounds['y'])))
    
    return u

### Определение линейного слоя с синусоидальной функцией активации

Пользуясь выводами из [статьи](https://arxiv.org/abs/2006.09661) и используя за основу представленный авторами [код](https://github.com/vsitzmann/siren) реализуем класс линейного слоя с синусоидальной функцией активации, как доработку линейного слоя.

`SinLayer` - класс слоя, представляющий собой слой нейронной сети с синусоидальной функцией активации.
- `__init__` - конструктор класса, который принимает параметры слоя.
- `init_weights()` - метод, который задает начальные значения весов.
- `forward` - метод,  который выполняет прямой проход через слой.

Этот слой используется в архитектуре нейросетей, таких как SIREN (Sinusoidal Representation Networks), где синусоидальная активация позволяет эффективно представлять сложные функции.

In [4]:
class SinLayer(torch.nn.Module):
    """
    Слой нейронной сети с синусоидальной функцией активации.

    Arguments
    ---------
    in_features: int
        Количество входных признаков.
    out_features: int
        Количество выходных признаков.
    bias: bool
        Использовать ли смещение в линейном слое.
    is_first: bool
        Является ли слой первым в сети.
    omega_0: float
        Частота для функции синуса.

    Methods
    -------
    init_weights
        Инициализирует веса слоя с учетом его положения в сети.
    forward: torch.tensor -> torch.tensor
        Прямой проход через слой, реализующий линейный слой и функцию активации.
    """
    def __init__(self, 
                 input: float, 
                 output: float, 
                 bias: bool=True,
                 is_first: bool=False, 
                 omega_0: float=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        self.in_features = input
        self.linear = torch.nn.Linear(input, output, bias=bias)
        self.init_weights()
    
    def init_weights(self):
        """
        Инициализация весов линейного слоя.
        """
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 
                                            1 / self.in_features)
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) 
                                            / self.omega_0, 
                                            np.sqrt(6 / self.in_features) 
                                            / self.omega_0)
        
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        """
        Прямой проход через слой.

        Parameters
        ----------
        input: torch.Tensor
            Входной тензор.

        Returns
        -------
        torch.Tensor
            Результат применения синусоидальной функции активации.
        """
        return torch.sin(self.omega_0 * self.linear(input))

### Определение модели PINN

`SinPINN` - класс модели, представляющий полносвязную физически-информированную нейронную сеть.
- `__init__` - конструктор класса, который инициализирует слои сети. Сеть состоит из нескольких линейных слоев `SinLayer`.
- `forward` - метод, который задаёт порядок прохождения входных данных через сеть. Этот метод принимает на вход тензоры `x`, `y` и `t` и возвращает предсказание модели (температуру).

In [6]:
class SinPINN(nn.Module):
    """
    Физически-информированная нейронная сеть (PINN) с функцией активации синус для моделирования теплопереноса.

    Arguments
    ---------
    layers: nn.ModuleList
        Список линейных слоёв нейронной сети.

    Methods
    -------
    forward: (torch.tensor, torch.tensor, torch.tensor) -> torch.tensor
        Прямой проход через сеть, возвращающий предсказанные значения температуры для входных данных x, y, t.
    """
    def __init__(self, 
                 layers: list):
        super().__init__()
        self.layers = nn.ModuleList()        
        for i in range(len(layers) - 1):
            if i == 0:
                self.layers.append(SinLayer(layers[i], 
                                             layers[i+1], is_first=True))
            elif i == len(layers) - 2:
                self.layers.append(nn.Linear(layers[i], 
                                             layers[i+1],))
            else:
                self.layers.append(SinLayer(layers[i], 
                                             layers[i+1], is_first=False))
    
    def forward(self, 
                x: torch.Tensor, 
                y: torch.Tensor, 
                t: torch.Tensor) -> torch.Tensor:
        """
        Прямой проход через сеть.

        Parameters
        ----------
        x: torch.Tensor
            Тензор координат x.
        y: torch.Tensor
            Тензор координат y.
        t: torch.Tensor
            Тензор время t.

        Returns
        -------
        torch.Tensor
            Выход сети.
        """
        inputs = torch.cat([x.reshape(-1, 1), 
                            y.reshape(-1, 1), 
                            t.reshape(-1, 1)], dim=1)
        for layer in self.layers:
            inputs = layer(inputs)
        return inputs

### Определение функции потерь

Функция потерь включает три компонента:
- **Потери на начальных условиях** – сравнение предсказанного значения температуры в момент времени $ t=0 $.
- **Потери на граничных условиях** – сравнение предсказанного значения температуры в точках $ (x, y) \in \partial \Omega $, где $ \Omega = [0,L] \times [0,L] $.
- **Потери для уравнения** – невязка, посчитанная по дифференциальному уравнению $ u_t - \alpha (u_{xx} + u_{yy})  = 0 $.

Для начала рассмотрим компонент потерь для уравнения и напишем функцию для его расчета.

In [7]:
def pde_residual(model: SinPINN, 
                 x: torch.Tensor, 
                 y: torch.Tensor, 
                 t: torch.Tensor, 
                 alpha: float):
    """
    Вычисляет остаток по уравнению теплопроводности для модели PINN.
    Вид уравнения:
        u_t - alpha * (u_xx + u_yy) = 0

    Parameters
    ----------
    model: torch.nn.Module
        Модель, вычисляющая u(x, y, t).
    x: torch.Tensor
        Тензор координаты x.
    y: torch.Tensor
        Тензор координаты y.
    t: torch.Tensor
        Тензор времени t.
    alpha: float
        Коэффициент теплопроводности.

    Returns
    -------
        torch.Tensor
            Остаток уравнения f = u_t - alpha*(u_xx + u_yy).
    """
    u = model(x, y, t)

    u_x, u_y, u_t = torch.autograd.grad(u, [x, y, t], 
                                        grad_outputs=torch.ones_like(u), 
                                        create_graph=True)
    
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), 
                               create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), 
                               create_graph=True)[0]
    
    f = u_t - alpha * (u_xx + u_yy)
    return f

### Подготовка данных

Для проверки выполнения уравнений задачи на предсказаниях модели, необходимо создать множество точке, на которых будет считаться функция потерь. Поскольку задача формулируется тремя уравнениями (уравнение теплопереноса, начальное и граничное условие) создадим множества точек коллокации, для каждого из уравнений соответственно.

In [9]:
def generate_pde_points(n: int, 
                        bounds: dict, 
                        device: torch.device):
    """
    Генерирует точки коллокации для уравнения PDE.

    Parameters
    ----------
    n: int
        Число точек коллокации.
    bounds: dict
        Границы области.
    device: torch.device
        Устройство (CPU или GPU).

    Returns
    -------
    Tuple[torch.Tensor, torch.Tensor, torch.Tensor]
        Тензоры x, y, t с requires_grad=True.
    """
    x = (bounds['x'] * torch.rand(n, 1, device=device)).requires_grad_()
    y = (bounds['y'] * torch.rand(n, 1, device=device)).requires_grad_()
    t = (bounds['t'] * torch.rand(n, 1, device=device)).requires_grad_()
    return x, y, t

In [10]:
def generate_initial_condition(n: int, 
                               bounds: dict, 
                               device: torch.device):
    """
    Генерирует точки для начального условия.

    Parameters
    ----------
    n: int
        Число точек начального условия.
    bounds: dict
        Границы расчетной области.
    device: torch.device
        Устройство (CPU или GPU).

    Returns
    -------
    Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        Тензоры x, y, t, u, 
        где t — нулевой тензор, а u = sin(pi * x/L) * sin(pi * y/L).
    """
    x = bounds['x'] * torch.rand(n, 1, device=device)
    y = bounds['y'] * torch.rand(n, 1, device=device)
    t = torch.zeros_like(x, device=device)
    u = torch.sin(np.pi * x/bounds['x']) * torch.sin(np.pi * y/bounds['y'])
    return x, y, t, u

In [77]:
def generate_boundary_conditions(n: int, 
                                 bounds: dict, 
                                 device: torch.device):
    """
    Генерирует точки граничных условий.

    Parameters
    ----------
    n_b: int 
        Число точек для граничных условий.
    bounds: dict
        Границы области, например, {'x': 1.0, 'y': 1.0, 't': 1.0}.
    device: torch.device
        Устройство (CPU или GPU).

    Returns
    -------
    Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        x_b, y_b, t_b, u_b, где u_b = 0.
    """
    x = bounds['x'] * torch.rand(n, 1, device=device)
    y = bounds['y'] * torch.rand(n, 1, device=device)
    t = bounds['t'] * torch.rand(n, 1, device=device)

    border_selector = torch.randint(0, 4, (n, 1), device=device)
    x[border_selector == 0] = 0.0           # Левая грань
    x[border_selector == 1] = bounds['x']   # Правая грань
    y[border_selector == 2] = 0.0           # Нижняя грань
    y[border_selector == 3] = bounds['y']   # Верхняя грань

    u_b = torch.zeros_like(t, device=device)
    return x, y, t, u_b

### Обучение модели

Чтобы достичь воспроизводимости результатов, зафиксируем случайную генерацию. Для ускорения последующих вычислений определим доступное устройство (GPU, если доступно, или CPU). 

Объявим параметры задачи, такие как число точек коллокации различных типов, границы расчетной области по пространству и времени, а также коэффициент температура проводности

In [78]:
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

N_F, N_B, N_I = 10000, 5000, 2000
BOUNDS = {'x': 0.1, 'y': 0.1, 't': 12.0}
ALPHA = 1.12 * 1e-4

model = SinPINN([3, 64, 1]).to(device)
mse_loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

x_f, y_f, t_f = generate_pde_points(N_F, BOUNDS, device)
x_i, y_i, t_i, u_i = generate_initial_condition(N_I, BOUNDS, device)
x_b, y_b, t_b, u_b = generate_boundary_conditions(N_B, BOUNDS, device)

w_pde, w_bc, w_ic = 10.0, 1.0, 1.0

Перейдем к обучению модели. Основной цикл обучения проходит в течение заданного количества эпох, где на каждой эпохе:
- Обнуляются градиенты.
- Вычисляются компоненты функции потерь.
- Производится обратное распространение ошибки.
- Обновляются параметры модели.
- Сохраняется история значений компонент функции потерь.
- Периодически выводится значение потерь для контроля процесса обучения.

In [ ]:
epochs = 20000

loss_history = {'pde': [],
                'ic': [],
                'bc': []}

for epoch in range(epochs+1):
    optimizer.zero_grad()
    
    u_pred_i = model(x_i, y_i, t_i)
    loss_ic = mse_loss(u_pred_i, u_i)
    
    u_pred_b = model(x_b, y_b, t_b)
    loss_bc = mse_loss(u_pred_b, u_b)
    
    loss_pde = mse_loss(pde_residual(model, x_f, y_f, t_f, ALPHA), 
                        torch.zeros_like(x_f))
    
    loss = w_ic * loss_ic + w_bc * loss_bc + w_pde * loss_pde
    
    loss.backward()
    optimizer.step()

    loss_history['ic'].append(loss_ic.item())
    loss_history['bc'].append(loss_bc.item())
    loss_history['pde'].append(loss_pde.item())
    
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.3e}, "
              f"IC: {loss_ic.item():.3e}, "
              f"BC: {loss_bc.item():.3e}, "
              f"PDE: {loss_pde.item():.3e}")

Для визуальной оценки качества процесса обучения можно провести визуализацию значений компонент функции потерь от эпох в логарифмической шкале.

In [ ]:
def plot_loss_history(loss_history: dict, 
                      step: int=200):
    """
    Функция для построения графика функции потерь.

    Parameters
    ----------
    loss_history : dict
        Словарь, содержащий данные для потерь.
    step : int, optional
        Шаг выборки данных для отображения, по умолчанию 100.
    """
    epochs = range(0, len(loss_history['pde']), step)

    plt.plot(epochs, loss_history['pde'][::step], label='PDE Loss')
    plt.plot(epochs, loss_history['bc'][::step], label='BC Loss')
    plt.plot(epochs, loss_history['ic'][::step], label='IC Loss')
    
    plt.yscale('log')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid()
    plt.legend()
    plt.show()


plot_loss_history(loss_history)

В дополнение к оптимизации методом `Adam` часто используется квазиньютоновский метод `LBFGS`. Он вычислительно сложнее, но хорошо подходит для квадратичных функций потерь, какую мы и используем, позволяя найти минимум за несколько итераций. Однако `LBFGS` склонен попадать в локальные минимумы при спуске, в то время как у `Adam` не такого недостатка. По этим причинам они используются совместно и именно в указанной последовательности.

Из-за несколько отличающегося интерфейса взаимодействия в реализации метода `LBFGS` в библиотеке `Pytorch` от остальных оптимизаторов, необходимо объявить вспомогательную функцию `closure`, которая будет вызываться при работе оптимизатора.

In [ ]:
def closure():
    """
    Вспомогательная функция для оптимизатора LBFGS.
    """
    optimizer_lbfgs.zero_grad()

    loss_ic = mse_loss(model(x_i, y_i, t_i), u_i)

    loss_bc = mse_loss(model(x_b, y_b, t_b), u_b)

    loss_pde = mse_loss(pde_residual(model, x_f, y_f, t_f, ALPHA), 
                           torch.zeros_like(x_f))
    
    loss = w_ic * loss_ic + w_bc * loss_bc + w_pde * loss_pde

    loss.backward()
    
    return loss

optimizer_lbfgs = optim.LBFGS(model.parameters(), lr=1.0, 
                              max_iter=500, history_size=50)
optimizer_lbfgs.step(closure)

Таким образом мы получили обученную модель для решения поставленной задачи. Проведем оценку полученного решения с помощью PINN сравнив его с объявленным ранее аналитическим решением.

In [ ]:
model.to(device)

N_test = 100
t_test = 1.0
x_test = np.linspace(0, BOUNDS['x'], N_test)
y_test = np.linspace(0, BOUNDS['y'], N_test)

X, Y = np.meshgrid(x_test, y_test)
X_test = torch.tensor(X.reshape(-1, 1), dtype=torch.float32, device=device)
Y_test = torch.tensor(Y.reshape(-1, 1), dtype=torch.float32, device=device)
T_test = torch.full_like(X_test, t_test, device=device)

with torch.no_grad():
    u_pred = model(X_test, Y_test, T_test).cpu().numpy()

u_exact = analytical_solution(X, Y, t_test, 
                              bounds=BOUNDS, alpha=ALPHA).reshape(-1, 1)

plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.contourf(X, Y, u_exact.reshape(N_test, N_test), 100, cmap='jet')
plt.title("Аналитическое решение")
plt.axis('equal')
plt.colorbar()

plt.subplot(1,3,2)
plt.contourf(X, Y, u_pred.reshape(N_test, N_test), 100, cmap='jet')
plt.title("Решение PINN")
plt.axis('equal')
plt.colorbar()

plt.subplot(1,3,3)
plt.contourf(X, Y, u_exact.reshape(N_test, N_test) - u_pred.reshape(N_test, N_test), 100, cmap='jet')
plt.title("Решение PINN")
plt.axis('equal')
plt.colorbar()

plt.show()

In [ ]:
def create_visualization(model: SinPINN, 
                         analytical_solution: callable, 
                         bounds: dict, 
                         alpha: float):
    """
    """
    x = np.linspace(0, bounds['x'], 100)
    y = np.linspace(0, bounds['y'], 100)
    mesh_x, mesh_y = np.meshgrid(x, y)
    
    pt_x = torch.tensor(mesh_x.ravel(), 
                        dtype=torch.float32).unsqueeze(1)
    pt_y = torch.tensor(mesh_y.ravel(), 
                        dtype=torch.float32).unsqueeze(1)
    
    time_steps = np.linspace(0, bounds['t'], 6)
    
    pinn_solutions, analytic_solutions = [], []
    with torch.no_grad():
        for t in time_steps:
            pt_t = torch.full_like(pt_x, t)           

            u_pred = model(pt_x, pt_y, pt_t).numpy().reshape(mesh_x.shape)
            pinn_solutions.append(u_pred)
            
            u_analyt = analytical_solution(mesh_x, mesh_y, t, bounds, alpha)
            analytic_solutions.append(u_analyt)
    
    vmin_pinn = min(arr.min() for arr in pinn_solutions)
    vmax_pinn = max(arr.max() for arr in pinn_solutions)
    
    vmin_analyt = min(arr.min() for arr in analytic_solutions)
    vmax_analyt = max(arr.max() for arr in analytic_solutions)

    vmin = min(vmin_pinn, vmin_analyt)
    vmax = max(vmax_pinn, vmax_analyt)
    
    fig, axes = plt.subplots(2, 6, figsize=(20, 8))

    for i, (pinn, analyt) in enumerate(zip(pinn_solutions, analytic_solutions)):
        ax_pinn = axes[0, i]
        im_pinn = ax_pinn.pcolormesh(mesh_x, mesh_y, pinn, 
                                     cmap='jet', 
                                     vmin=vmin, 
                                     vmax=vmax)
        ax_pinn.set_title(f'Time = {time_steps[i]:.2f}')
        ax_pinn.set_xticks([])
        ax_pinn.set_yticks([])
        
        ax_analyt = axes[1, i]
        im_analyt = ax_analyt.pcolormesh(mesh_x, mesh_y, analyt, 
                                         cmap='jet', 
                                         vmin=vmin, 
                                         vmax=vmax)
        ax_analyt.set_xticks([])
        ax_analyt.set_yticks([])
    
    plt.tight_layout()
    
    return fig

model.to('cpu')
fig = create_visualization(model, analytical_solution, bounds=BOUNDS, alpha=ALPHA)
plt.show()

### Экспорт модели в формат ONNX

После обучения модель переводится в режим двойной точности с помощью метода `double()`. Затем, используя шаблон входного тензора, модель экспортируется в формат ONNX. При этом задаются имена входных и выходных параметров для корректной интеграции.  

In [ ]:
model.double()

x_temp = torch.randn(1, 1, device=device, dtype=torch.float64) * BOUNDS['x']
y_temp = torch.randn(1, 1, device=device, dtype=torch.float64) * BOUNDS['y']
t_temp = torch.randn(1, 1, device=device, dtype=torch.float64) * BOUNDS['t']

input_names = ['x', 'y', 't']
output_names = ['u']

torch.onnx.export(model,
                  (x_temp, y_temp, t_temp),
                  "2DheatModel.onnx",
                  opset_version=12,
                  input_names=input_names,
                  output_names=output_names)

### Добавление метаданных о границах парамеров

К сожалению, функция `torch.onnx.export` не позволяет передать дополнительные данные о граничных значениях параметров, в пределах которых проводился расчет. Для их добавления необходимо загрузить ранее экспортированную модель из файла, создать словарь метаданных, содержащий информацию о границах.

In [ ]:
onnx_model = onnx.load("2DheatModel.onnx")

metadata_dict = {"x": f"{BOUNDS['x']}",
                 "y": f"{BOUNDS['y']}",
                 "t": f"{BOUNDS['t']}"}

metadata_json = json.dumps(metadata_dict)

metadata_prop = onnx.StringStringEntryProto()
metadata_prop.key = "bounds"
metadata_prop.value = metadata_json

onnx_model.metadata_props.append(metadata_prop)
onnx.save(onnx_model, "2DheatModel.onnx")

### Проверка файла ONNX

Опционально, можно проверить правильность структуры модели, параметров и метаданных. Для этого может использоваться библиотека `Netron`, которая визуализирует структуру модели.

In [ ]:
onnx_model_path = "2DheatModel.onnx"

url = netron.start(onnx_model_path, browse=False)
IFrame(src=f"http://localhost:8080/", width=600, height=600)

In [ ]:
netron.stop()